In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import pandas as pd
import numpy as np

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', context=ctx).read()

In [4]:
soup = BeautifulSoup(html, "html.parser")

### Get table from soup and convert to dataframe

In [5]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
#df.columns = df.iloc[0]
#df = df[1:]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove boroughs with status 'Not assigned'

In [6]:
filter = df["Borough"] != "Not assigned"
df = df[filter]

### Change 'Not assigned' neighbourhood to applicable borough

In [7]:
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']

### Group postcodes together and combine neighbourhoods

In [8]:
df = df.groupby(['Postcode','Borough']).agg(lambda Neighbourhood: ', '.join(Neighbourhood))
df = df.reset_index()

In [9]:
df.shape

(103, 3)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


###  Read in Geospatial_Coordinates.csv

In [11]:
df_latlong = pd.read_csv('Geospatial_Coordinates.csv')
df_latlong = df_latlong.rename(columns = {'Postal Code':'Postcode'})
df_latlong.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge dataframe containing Wikipedia information (df) with dataframe of lat/long info from csv (df_latlong)

In [12]:
df = pd.merge(df, df_latlong, on = 'Postcode', how = 'left')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df.shape

(103, 5)

In [14]:
df.to_csv('Toronto_latlong.csv')